# Django2.2

**Python Web Framework**：<https://wiki.python.org/moin/WebFrameworks>

先说句大实话，Web端我一直都是`Net技术站`的`MVC and WebAPI`，Python我一直都是用些数据相关的知识（爬虫、简单的数据分析等）Web这块只是会Flask，其他框架也都没怎么接触过，都说`Python`的`Django`是`建站神器`，有`自动生成后端管理页面`的功能，于是乎就接触了下`Django2.2`（目前最新版本）
> 逆天点评：Net的MVC最擅长的就是（通过Model+View）`快速生成前端页面和对应的验证`，而Python的`Django`最擅长的就是（通过注册Model）`快速生成后台管理页面`。**这两个语言都是快速建站的常用编程语言**（项目 V1~V2 阶段）

网上基本上都是Django1.x的教程，很多东西在2下都有点不适用，所以简单记录下我的学习笔记以及一些心得：
> PS：ASP.Net MVC相关文章可以参考我16年写的文章：<https://www.cnblogs.com/dunitian/tag/MVC/>

官方文档：<https://docs.djangoproject.com/zh-hans/2.2/releases/2.2/>

## 1.环境

### 1.虚拟环境

这个之前的确没太大概念，我一直都是用Conda来管理不同版本的包，现在借助Python生态圈里的工具`virtualenv`和`virtualenvwapper`












---

### 2.Django命令

1.**创建一个空项目：`django-admin startproject 项目名称`**
> PS：项目名不要以数字开头哦~

```shell
# 创建一个base_demo的项目
django-admin startproject base_demo

# 目录结构
|-base_demo （文件夹）
|---__init__.py（说明这个文件夹是一个Python包）
|---settings.py（项目配置文件：创建应用|模块后进行配置）
|---urls.py（URL路由配置）
|---wsgi.py（遵循wsgi协议：web服务器和Django的交互入口）
|-manage.py（项目管理文件，用来生成应用|模块）
```

2.**创建一个应用：`python manage.py startapp 应用名称`**
> 项目中一个模块就是一个应用，eg：商品模块、订单模块等

```shell
# 创建一个用户模块
python manage.py startapp users

├─base_demo
│    __init__.py
│    settings.py
│    urls.py
│    wsgi.py
├─manage.py（项目管理文件，用来生成应用|模块）
│
└─users（新建的模块|应用）
│   │  __init__.py
│   │  admin.py（后台管理相关）
│   │  models.py（数据库相关模型）
│   │  views.py（相当于MVC中的C，用来定义处理|视图函数）
│   │  tests.py（写测试代码）
│   │  apps.py：配置应用的元数据（可选）
│   │
│   └─migrations：数据迁移模块（根据Model内容生成的）
│      __init__.py
```

**PS：记得在项目(`base_demo`)的settings.py注册一下应用模块哦~**

```py
INSTALLED_APPS = [
    ......
    'users', # 注册自己创建的模块|应用
]
```

3.**运行项目：`python manage.py runserver`**
> PS：指定端口：`python manage.py runserver 8080`

## 2.MVT入门

**大家都知道MVC（模型-视图-控制器），而Django的MVC叫做MVT（模型-视图-模版）**
> PS：Django出来很早，名字是自己定义的，用法和理念是一样的

### 2.1.M（模型）

#### 2.1.1.类的定义

- 1.**生成迁移文件：`python manage.py makemigrations`**
    - PS：根据编写好的Model文件生成（模型里面可以不用定义ID属性）
- 2.**执行迁移生成表：`python mange.py migrate`**
    - PS：执行生成的迁移文件

PS：类似于EF的`CodeFirst`，Django默认使用的是`sqlite`，更改数据库后面会说的

先看个演示案例：

**1.定义类文件**（会根据Code来生成DB）
```py
# users > models.py

from django.db import models

# 用户信息表
class UserInfo(models.Model):
    # 字符串类型，最大长度为20
    name = models.CharField(max_length=20)
    # 创建时间：日期类型
    create_time = models.DateTimeField()
    # 更新时间
    update_time = models.DateTimeField()
```

**2. 生成数据库**
```shell
# 生成迁移文件
> python manage.py makemigrations

Migrations for 'userinfo':
  userinfo\migrations\0001_initial.py
    - Create model UserInfo

# 执行迁移生成表
> python manage.py migrate
Operations to perform:
  Apply all migrations: admin, auth, contenttypes, sessions, userinfo
Running migrations:
  Applying contenttypes.0001_initial... OK
  Applying auth.0001_initial... OK
  Applying admin.0001_initial... OK
  Applying admin.0002_logentry_remove_auto_add... OK
  Applying admin.0003_logentry_add_action_flag_choices... OK
  Applying contenttypes.0002_remove_content_type_name... OK
  Applying auth.0002_alter_permission_name_max_length... OK
  Applying auth.0003_alter_user_email_max_length... OK
  Applying auth.0004_alter_user_username_opts... OK
  Applying auth.0005_alter_user_last_login_null... OK
  Applying auth.0006_require_contenttypes_0002... OK
  Applying auth.0007_alter_validators_add_error_messages... OK
  Applying auth.0008_alter_user_username_max_length... OK
  Applying auth.0009_alter_user_last_name_max_length... OK
  Applying sessions.0001_initial... OK
  Applying userinfo.0001_initial... OK
```

然后就自动生成对应的表了 ==> **`users_userinfo`**（应用名_模块中的类名）

![1.sqlite.png](https://img2018.cnblogs.com/blog/1127869/201904/1127869-20190425132159094-1430139515.png)

知识拓展：默认时间相关文章：<https://www.cnblogs.com/huchong/p/7895263.html>

####  2.1.2.生成后台

##### 1.配置本地化（设置后台管理页面是中文）

主要就是修改`settings.py`文件的`语言`和`时区`（后台管理的语言和时间）

```py
# 使用中文（zh-hans可以这么记==>zh-汉'字'）
LANGUAGE_CODE = 'zh-hans'

# 设置中国时间
TIME_ZONE = 'Asia/Shanghai'
```

##### 2.创建管理员

**创建系统管理员：`python manage.py createsuperuser`**

```shell
python manage.py createsuperuser
用户名 (leave blank to use 'win10'): dnt # 如果不填，默认是计算机用户名
电子邮件地址: # 可以不设置
Password:
Password (again):
Superuser created successfully.
```

**经验：如果忘记密码可以创建一个新管理员账号，然后把旧的删掉就行了**
> PS：根据新password字段，修改下旧账号的password也可以

课后拓展：<a href="https://blog.csdn.net/dsjakezhou/article/details/84319228">修改django后台管理员密码</a>

##### 3.后台管理页面

主要就是**在admin中注册模型类**

比如给之前创建的UserInfo类创建对应的管理页面：

```py
# base_demo > users > admin.py

from users.models import UserInfo
# from .models import UserInfo

# 注册模型类（自动生成后台管理页面）
admin.site.register(UserInfo) # .site别忘记
```

然后运行Django（`python manage.py runserver`），访问"127.0.0.1:8080/admin"，登录后就就可以管理了
> PS：如果不想交admin，而是想在root下。那么可以修改项目的`urls.py`（后面会说）

![2.admin.png](https://img2018.cnblogs.com/blog/1127869/201904/1127869-20190425132701645-2058965524.png)

##### 4.制定化显示

注册模型类就ok了，但是显示稍微有点不人性化，eg：

![3.添加用户.png](https://img2018.cnblogs.com/blog/1127869/201904/1127869-20190425133243297-1194373344.png)

列表页显示出来的标题是UserInfo对象，而我们平时一般显示用户名等信息
![3.列表.png](https://img2018.cnblogs.com/blog/1127869/201904/1127869-20190425133430777-259130787.png)

so ==> 可以自己改写下

回顾下之前讲的：（程序是显示的`str(对象)`，那么我们重写魔方方法`__str__`即可改写显示了）

```py
# base_demo > users > models.py

# 用户信息表
class UserInfo(models.Model):
    # 字符串类型，最大长度为20
    name = models.CharField(max_length=20)
    # 创建时间：日期类型
    create_time = models.DateTimeField()
    # 更新时间
    update_time = models.DateTimeField()

    def __str__(self):
        """为了后台管理页面的美化"""
        return self.name
```

这时候再访问就美化了：(**不用重启Django**）

![3.美化列表.png](https://img2018.cnblogs.com/blog/1127869/201904/1127869-20190425133840428-1396356170.png)

Django就没有提供对应的方法？NoNoNo，我们继续看：

```py
# base_demo > users > admin.py

from .models import UserInfo

# 自定义模型管理页面
class UserInfoAdmin(admin.ModelAdmin):
    # 自定义管理页面的列表显示字段（和类属性相对应）
    list_display = ["id", "name", "create_time", "update_time", "datastatus"]

# 注册模型类和模型管理类（自动生成后台管理页面）
admin.site.register(UserInfo, UserInfoAdmin)
```

其他什么都不用修改，后端管理列表的布局就更新了：
> PS：设置Model的`verbose_name`就可以在后台显示中文，eg：`name = models.CharField(max_length=25, verbose_name="姓名")`

![3.自定义列表页面.png](https://img2018.cnblogs.com/blog/1127869/201904/1127869-20190425140525987-1801746184.png)

还有更多个性化的内容后面会继续说的~

### 2.3.V（视图）

这个类比于MVC的C（控制器）
> PS：这块比Net的MVC和Python的Flask要麻烦点，url地址要简单配置下映射关系（小意思，不花太多时间）

这块刚接触稍微有点绕，所以我们借助图来看：

**比如我们想访问users应用下的首页（`/users/index`）**

#### 2.3.1.设置视图函数

这个和定义控制器里面的方法没区别：
> PS：函数必须含`request`（类比下类方法必须含的self）

![4.users模块的视图.png](https://img2018.cnblogs.com/blog/1127869/201904/1127869-20190426151450564-1184327950.png)

```py
from django.http import HttpResponse

# 1.定义视图函数
# http://127.0.0.1:8000/users/index
def index(request):
    print(request)
    # 响应浏览器请求（需要页面就去T拿，需要数据就去M找）
    return HttpResponse('这是users应用模块的index页面哦~')
```

#### 2.3.2.配置路由

因为我想要的地址是：`/users/index`，那么我在项目urls中也需要配置下访问`/users`的路由规则：
> PS：我是防止以后模块多了管理麻烦，所以分开写，要是你只想在一个urls中配置也无妨

![4.配置users路由.png](https://img2018.cnblogs.com/blog/1127869/201904/1127869-20190426152042117-501509824.png)

```py
# base_demo > urls.py

from django.contrib import admin
from django.urls import path, include

# 项目urls配置文件
urlpatterns = [
    path('users/', include("users.urls")),  # 配置项
]
```

最后再贴一下users应用模块的匹配：

![4.users模块下视图函数.png](https://img2018.cnblogs.com/blog/1127869/201904/1127869-20190426152347506-1754124621.png)

```py
# users > urls.py

from django.urls import path
from . import views

# 2.进行url配置（建立url地址和视图的对应关系）
urlpatterns = [
    # /users/index ==> view的index处理函数
    path('index', views.index),
]
```

#### 2.3.3.url访问

这时候你访问`127.0.0.1:8000/users/index`就可以了：

![4.页面.png](https://img2018.cnblogs.com/blog/1127869/201904/1127869-20190426154047862-2133863089.png)

简单说下这个过程：

1. 先去项目的urls.py中进行匹配
    - `path('users/', include("users.urls")),  # 配置项`
2. 发现只要是以`/users/`开头的都使用了`users`模块自己的`urls.py`来匹配
    - `path('index', views.index),`
3. 发现访问`/users/index`最后进入的视图函数是`index`
4. 然后执行`def index(request):pass`里面的内容并返回

### 2.4.T（模版）

这个类比于MVC的V，我们来看个简单案例：

#### 2.4.1.创建模版

Django1.x版本需要配置下模版路径之类的，现在只要在对应模块下创建`templates`文件夹就可以直接访问了

我们来定义一个list的模版：

![5.定义模版.png](https://img2018.cnblogs.com/blog/1127869/201904/1127869-20190427143442068-1168076141.png)

定义视图函数（类比定义控制器方法）

![5.定义视图函数.png](https://img2018.cnblogs.com/blog/1127869/201904/1127869-20190427144206441-712238444.png)

配置对应的路由：

![5.配置对应的url.png](https://img2018.cnblogs.com/blog/1127869/201904/1127869-20190426215910537-278042489.png)

然后就出效果了：

![5.效果.png](https://img2018.cnblogs.com/blog/1127869/201904/1127869-20190426220018043-1314480230.png)

如果把之前添加的数据删除掉，也会显示默认效果：

![5.没有内容.png](https://img2018.cnblogs.com/blog/1127869/201904/1127869-20190427144518455-203345918.png)

#### 2.4.2.指定模版

也可以指定模版位置：（看个人习惯）

打开项目`settings.py`文件，设置`TEMPLATES`的`DIRS`值，来指定默认模版路径：

```py
# Base_dir：当前项目的绝对路径
# BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))

# https://docs.djangoproject.com/zh-hans/2.2/ref/settings/#templates
TEMPLATES = [
    {
        ...
        'DIRS': [os.path.join(BASE_DIR, 'templates')], # 模版文件的绝对路径
        ...
    },
]
```

### 扩展：使用MySQL数据库

这篇详细流程可以查看之前写的文章：<a href="" title="https://www.cnblogs.com/dotnetcrazy/p/10782441.html" target="_blank">稍微记录下Django2.2使用MariaDB和MySQL遇到的坑</a>

这边简单过下即可：

#### 1.创建数据库

Django不会帮你创建数据库，需要自己创建，eg：`create database django charset=utf8;`

#### 2.配置数据库

我把对应的文档url也贴了：

```py
# https://docs.djangoproject.com/en/2.2/ref/settings/#databases
DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.mysql',
        'NAME': 'django',  # 使用哪个数据库
        'USER': 'root',  # mysql的用户名
        'PASSWORD': 'dntdnt',  # 用户名对应的密码
        'HOST': '127.0.0.1',  # 数据库服务的ip地址
        'PORT': 3306,  # 对应的端口
        # https://docs.djangoproject.com/en/2.2/ref/settings/#std:setting-OPTIONS
        'OPTIONS': {
            # https://docs.djangoproject.com/zh-hans/2.2/ref/databases/#setting-sql-mode
            # SQLMode可以看我之前写的文章：https://www.cnblogs.com/dotnetcrazy/p/10374091.html
            'init_command': "SET sql_mode='STRICT_TRANS_TABLES'",  # 设置SQL_Model
        },
    }
}
```

最小配置：

![6.1.配置.png](https://img2018.cnblogs.com/blog/1127869/201904/1127869-20190428100909946-1594638238.png)

项目init.py文件中配置：

```py
import pymysql

# Django使用的MySQLdb对Python3支持力度不够，我们用PyMySQL来代替
pymysql.install_as_MySQLdb()
```

图示：

![6.2.init.png](https://img2018.cnblogs.com/blog/1127869/201904/1127869-20190428101008312-85199351.png)

#### 3.解决干扰

如果你的Django是最新的2.2,PyMySQL也是最新的0.93的话，你会发现Django会报错：
> django.core.exceptions.ImproperlyConfigured: mysqlclient 1.3.13 or newer is required; you have 0.9.3.

这个是Django对MySQLdb版本的限制，我们使用的是PyMySQL，所以不用管它

![6.3.错误1.png](https://img2018.cnblogs.com/blog/1127869/201904/1127869-20190428101746288-1699408933.png)

再继续运行发现又冒了个错误：`AttributeError: 'str' object has no attribute 'decode'`

这个就不能乱改了，所以先调试输出下：

![6.4.调试.png](https://img2018.cnblogs.com/blog/1127869/201904/1127869-20190428101925741-707545580.png)

发现是对字符串进行了decode解码操作：（一般对字符串进行编码，二进制进行解码）

![6.5.bug.png](https://img2018.cnblogs.com/blog/1127869/201904/1127869-20190428101950198-1132075286.png)

解决也很简单，改成encode即可

![6.6.ok.png](https://img2018.cnblogs.com/blog/1127869/201904/1127869-20190428102045457-1853588866.png)

然后就没问题了，之后创建新项目也不会有问题了

### 扩展：避免命令忘记

![7.偷懒.png](https://img2018.cnblogs.com/blog/1127869/201905/1127869-20190501115218198-957846270.png)

![7.智能提示.png](https://img2018.cnblogs.com/blog/1127869/201905/1127869-20190501115811402-1737176844.png)


## 3.MVT基础

### 3.1.M基础

#### 3.1.模型的定义

模型类的命名规则基本上和变量命名一致，然后添加一条：**不能含`__`**（双下划线）
> PS：这个后面讲查询的时候你就秒懂了（`__`来间隔关键词）



这边简单罗列下**常用字段类型**：


模版中动态生成URL地址，类似于Net里面的`@Url.Action("Edit","Home",new {id=13})`
> <https://docs.djangoproject.com/zh-hans/2.2/intro/tutorial03/#removing-hardcoded-urls-in-templates>

#### 扩展：指定表名
class xxx(models.Mode):
    class Meta:
        tb_name = "xx"

#### 3.2.ORM基础

https://docs.djangoproject.com/zh-hans/2.2/ref/models/querysets/

#### 执行SQL语句

官方文档：<https://docs.djangoproject.com/zh-hans/2.2/topics/db/sql/>

#### 扩展：查看生成SQL

课后拓展：<https://www.jianshu.com/p/b69a7321a115>

#### 3.3.模型管理器类

1. 改变查询的结果集
    - eg：程序里面都是假删除，而默认的`all()`把那些假删除的数据也查询出来了
2. 添加额外的方法
    - eg：



增（有连接关系的情况）

删（逻辑删除、删）

改（内连接关联修改）

查（总数、条件查询据、分页查询）

## 4.Admin后台

上面演示了一些简单的制定化知识点：<a href="#2.1.2.生成后台">上节回顾</a>，现在简单归纳下`Django2.2`admin相关设置：

### 4.1.修改后台管理页面的标题

大致效果如下：

![7.后台管理页面](https://img2018.cnblogs.com/blog/1127869/201905/1127869-20190504205940102-1264624779.png)

在`admin.py`中设置`admin.site.site_header`和`admin.site.site_title`：

![7.admin配置.png](https://img2018.cnblogs.com/blog/1127869/201905/1127869-20190504210121154-1717736601.png)

### 4.2.修改app在Admin后台显示的名称

大致效果如下：

![7.大致效果.png](https://img2018.cnblogs.com/blog/1127869/201905/1127869-20190504210312122-731544995.png)

先设置应用模块的中文名：**`verbose_name = 'xxx'`**

![7.设置应用模块的中文名.png](https://img2018.cnblogs.com/blog/1127869/201905/1127869-20190504204034852-45175516.png)

让配置生效：**`default_app_config = '应用名.apps.应用名Config'`**

![7.1.配置生效.png](https://img2018.cnblogs.com/blog/1127869/201905/1127869-20190504204327495-1189085248.png)

### 4.3.汉化显示应用子项

大致效果如下：

![7.2.效果.png](https://img2018.cnblogs.com/blog/1127869/201905/1127869-20190504210405885-72000235.png)

在每个模型类中设置**`Meta`**类，并设置`verbose_name`和`verbose_name_plural`

![7.2.汉化显示应用子项.png](https://img2018.cnblogs.com/blog/1127869/201905/1127869-20190504211207183-48476581.png)

### 4.4.汉化表单字段和提示

大致效果如下：

![7.3.效果.png](https://img2018.cnblogs.com/blog/1127869/201905/1127869-20190504211720982-363096327.png)

汉化表单的字段：**`verbose_name`**，显示字段提示：**`help_text`**

![7.3.汉化表单字段.png](https://img2018.cnblogs.com/blog/1127869/201905/1127869-20190504211623827-1855804997.png)

### 4.5.

列表显示

状态显示+字体颜色

文件上传

文本验证

Tag过滤


apt install sqliteman